### Word Group In Youtube

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 40000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:40000
file_ext:6
unique_word_ratio:90
include_wordgroup_ratio:90
word_usage_min:5
word_usage_max:100


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [7]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [8]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [9]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,لا,5782309
1,من,5493373
2,أن,4348055
3,في,4324498
4,هذا,4155278
...,...,...
686322,مساميّ,5
686323,الساندينيين,5
686324,تلاطفيني,5
686325,مساندوك,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,لا,5782309
1,من,5493373
2,أن,4348055
3,في,4324498
4,هذا,4155278
...,...,...
39995,فعليكم,464
39996,يطل,464
39997,يوماَ,464
39998,طعامًا,464


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [15]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

40000

#### Youtube Sentence Analysis

In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:04.000,مرحبا أصدقائي,Qx98dvR61cY
1,00:00:06.000,00:00:09.017,مضى اكثر من اسبوعين,Qx98dvR61cY
2,00:00:09.017,00:00:11.517,عندما هطلت الأمطار,Qx98dvR61cY
3,00:00:11.550,00:00:13.667,وحدثت فيضانات,Qx98dvR61cY
4,00:00:13.684,00:00:20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY
...,...,...,...,...
918116,00:00:34.396,00:00:36.396,قولتله خلاص سيبها,Hp54ZaJS1nI
918117,00:00:36.396,00:00:40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI
918118,00:00:45.588,00:00:48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI
918119,00:00:48.835,00:00:50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY
...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI


In [19]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [20]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [21]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,100.0,,"أصدقائي, مرحبا"
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,100.0,,"مضى, اسبوعين, اكثر, من"
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,66.6,هطلت,"عندما, الأمطار"
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY,0.0,"وحدثت, فيضانات",
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY,42.9,"والشائع, طيني, السيول, اصفر","لون, المعروف, أن"
...,...,...,...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI,33.3,"سيبها, قولتله",خلاص
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,50.0,"شايفين, حضنتها, مانتم","زى, كده, و"
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,62.5,"جامدة, حضنى, حسيت","بمشاعر, هى, جدا, و, فى"
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,75.0,شايل,"كأنك, بالظبط, طفل"


In [22]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,100.0,NaN,"أصدقائي, مرحبا"
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,100.0,NaN,"مضى, اسبوعين, اكثر, من"
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,66.6,هطلت,"عندما, الأمطار"
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY,0.0,"وحدثت, فيضانات",NaN
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY,42.9,"والشائع, طيني, السيول, اصفر","لون, المعروف, أن"
...,...,...,...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI,33.3,"سيبها, قولتله",خلاص
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,50.0,"شايفين, حضنتها, مانتم","زى, كده, و"
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,62.5,"جامدة, حضنى, حسيت","بمشاعر, هى, جدا, و, فى"
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,75.0,شايل,"كأنك, بالظبط, طفل"


In [23]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,100.0,NaN,"أصدقائي, مرحبا"
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,100.0,NaN,"مضى, اسبوعين, اكثر, من"
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,66.6,هطلت,"عندما, الأمطار"
3,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY,42.9,"والشائع, طيني, السيول, اصفر","لون, المعروف, أن"
4,5,22.667,27.667,وهذه السيول ذهبت للنهر قبل أسبوعين,Qx98dvR61cY,83.3,السيول,"للنهر, وهذه, قبل, أسبوعين, ذهبت"
...,...,...,...,...,...,...,...,...
896499,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI,33.3,"سيبها, قولتله",خلاص
896500,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,50.0,"شايفين, حضنتها, مانتم","زى, كده, و"
896501,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,62.5,"جامدة, حضنى, حسيت","بمشاعر, هى, جدا, و, فى"
896502,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,75.0,شايل,"كأنك, بالظبط, طفل"


In [24]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-24-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
5,6,27.667,34.434,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المي...,Qx98dvR61cY,83.3,"فلتر, نقيه","المياه, ولكن, تتحرك, بقت, كأنها, مياه, الغريب,...",ولكن الغريب بقت مياه تتحرك في الوادي وهذه المياه
6,7,40.434,44.400,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY,100.0,NaN,"المياه, يعرف, اين, من, تأتي, لا, وهذه, أحد",وهذه المياه من اين تأتي لا أحد يعرف
10,11,0.725,2.285,أهلا وسهلا فيكم وين ما كنتو,zRkO1P6_fZM,83.3,كنتو,"فيكم, ما, وين, أهلا, وسهلا",أهلا وسهلا فيكم وين ما
14,15,11.180,14.260,المطار الرئيسي في المنطقة الشرقية,zRkO1P6_fZM,100.0,NaN,"المطار, المنطقة, الرئيسي, في, الشرقية",المطار الرئيسي في المنطقة الشرقية
34,35,0.000,12.000,إذا أعجبك الفيديو و أحببت أن تشكرنا شارك بالقن...,nngU8qS7kBs,75.0,"تشكرنا, بالقناة","شارك, أن, الفيديو, أعجبك, شكرا, و, إذا, أحببت, لك",إذا أعجبك الفيديو و أحببت أن
...,...,...,...,...,...,...,...,...,...
896430,918029,128.047,130.547,في حق نفسه وما حسب,_bodYtWV23A,100.0,NaN,"نفسه, وما, حق, في, حسب",في حق نفسه وما حسب
896432,918031,133.795,136.653,في حق نفسه وما حسب,_bodYtWV23A,100.0,NaN,"نفسه, وما, حق, في, حسب",في حق نفسه وما حسب
896454,918062,421.895,424.485,غلط وما كان يقصد شي,_bodYtWV23A,100.0,NaN,"يقصد, كان, وما, غلط, شي",غلط وما كان يقصد شي
896494,918111,20.970,24.223,طاقة اللى جوايا ناحية الحيوانات عمرها ما تخلص,Hp54ZaJS1nI,87.5,جوايا,"تخلص, عمرها, الحيوانات, ما, اللى, ناحية, طاقة",ناحية الحيوانات عمرها ما تخلص


In [25]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المياه,27.667000,32.847984,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المي...,Qx98dvR61cY
1,وهذه المياه من اين تأتي لا أحد يعرف,40.434000,44.400000,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY
2,أهلا وسهلا فيكم وين ما,0.725000,2.053889,أهلا وسهلا فيكم وين ما كنتو,zRkO1P6_fZM
3,المطار الرئيسي في المنطقة الشرقية,11.180000,14.260000,المطار الرئيسي في المنطقة الشرقية,zRkO1P6_fZM
4,إذا أعجبك الفيديو و أحببت أن,0.000000,6.000000,إذا أعجبك الفيديو و أحببت أن تشكرنا شارك بالقن...,nngU8qS7kBs
...,...,...,...,...,...
327339,في حق نفسه وما حسب,128.047000,130.547000,في حق نفسه وما حسب,_bodYtWV23A
327340,في حق نفسه وما حسب,133.795000,136.653000,في حق نفسه وما حسب,_bodYtWV23A
327341,غلط وما كان يقصد شي,421.895000,424.485000,غلط وما كان يقصد شي,_bodYtWV23A
327342,ناحية الحيوانات عمرها ما تخلص,22.054333,24.223000,طاقة اللى جوايا ناحية الحيوانات عمرها ما تخلص,Hp54ZaJS1nI


In [26]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المياه,27.367000,33.147984,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المي...,Qx98dvR61cY
1,وهذه المياه من اين تأتي لا أحد يعرف,40.134000,44.700000,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY
2,أهلا وسهلا فيكم وين ما,0.425000,2.353889,أهلا وسهلا فيكم وين ما كنتو,zRkO1P6_fZM
3,المطار الرئيسي في المنطقة الشرقية,10.880000,14.560000,المطار الرئيسي في المنطقة الشرقية,zRkO1P6_fZM
4,إذا أعجبك الفيديو و أحببت أن,-0.300000,6.300000,إذا أعجبك الفيديو و أحببت أن تشكرنا شارك بالقن...,nngU8qS7kBs
...,...,...,...,...,...
327339,في حق نفسه وما حسب,127.747000,130.847000,في حق نفسه وما حسب,_bodYtWV23A
327340,في حق نفسه وما حسب,133.495000,136.953000,في حق نفسه وما حسب,_bodYtWV23A
327341,غلط وما كان يقصد شي,421.595000,424.785000,غلط وما كان يقصد شي,_bodYtWV23A
327342,ناحية الحيوانات عمرها ما تخلص,21.754333,24.523000,طاقة اللى جوايا ناحية الحيوانات عمرها ما تخلص,Hp54ZaJS1nI


In [27]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المياه,27,33,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المي...,Qx98dvR61cY
1,وهذه المياه من اين تأتي لا أحد يعرف,40,45,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY
2,أهلا وسهلا فيكم وين ما,0,2,أهلا وسهلا فيكم وين ما كنتو,zRkO1P6_fZM
3,المطار الرئيسي في المنطقة الشرقية,11,15,المطار الرئيسي في المنطقة الشرقية,zRkO1P6_fZM
4,إذا أعجبك الفيديو و أحببت أن,0,6,إذا أعجبك الفيديو و أحببت أن تشكرنا شارك بالقن...,nngU8qS7kBs
...,...,...,...,...,...
327339,في حق نفسه وما حسب,128,131,في حق نفسه وما حسب,_bodYtWV23A
327340,في حق نفسه وما حسب,133,137,في حق نفسه وما حسب,_bodYtWV23A
327341,غلط وما كان يقصد شي,422,425,غلط وما كان يقصد شي,_bodYtWV23A
327342,ناحية الحيوانات عمرها ما تخلص,22,25,طاقة اللى جوايا ناحية الحيوانات عمرها ما تخلص,Hp54ZaJS1nI


In [28]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,هناك دائما طريقة صحيحة للقيام بالأشياء في رأسك...,209,239,هناك دائما طريقة صحيحة للقيام بالأشياء في رأسك...,WcKiIL31tqY
1,هذا بشكل مثالي الأمر لا يتعلق بعدد المرات التي...,491,523,هناك اقتباس يلخص هذا بشكل مثالي الأمر لا يتعلق...,TpP4LigaLWM
2,ماذا سيحدث إذا قمت بإجراء هذه التغييرات الكبير...,80,109,ماذا سيحدث إذا قمت بإجراء هذه التغييرات الكبير...,tEuQpz6cC0U
3,أو ثلاث وظائف مختلفة لمحاولة تغطية الفواتير وم...,115,138,غالبا ما يعمل الفقراء في وظيفتين أو ثلاث وظائف...,PX6Df4I95wA
4,الطريقة التي يتواصل بها الشخص معك بالعين يمكن ...,186,211,الطريقة التي يتواصل بها الشخص معك بالعين يمكن ...,hVXlKj9gahs
...,...,...,...,...,...
327339,ـ ـ ـ ـ ـ,65,70,ـ ـ ـ ـ ـ,PlpxSN2qDfg
327340,م ن ش ي ء,1360,1363,فما أغنى عنهم سمعهم ولا أبصارهم ولا أفئدتهم مِ...,GxKz46WcZGU
327341,م ن ش ي ء,1493,1496,فما أغنى عنهم سمعهم ولا أبصارهم ولا أفئدتهم مِ...,GxKz46WcZGU
327342,ب ع ل ى غ,1861,1868,أن ينزل الله من فضله على من يشاء من عباده فباء...,QNxG9RTbDBk


In [29]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,هناك دائما طريقة صحيحة للقيام بالأشياء في رأسك...,209,239,هناك دائما طريقة صحيحة للقيام بالأشياء في رأسك...,WcKiIL31tqY,87
1,هذا بشكل مثالي الأمر لا يتعلق بعدد المرات التي...,491,523,هناك اقتباس يلخص هذا بشكل مثالي الأمر لا يتعلق...,TpP4LigaLWM,89
2,ماذا سيحدث إذا قمت بإجراء هذه التغييرات الكبير...,80,109,ماذا سيحدث إذا قمت بإجراء هذه التغييرات الكبير...,tEuQpz6cC0U,85
3,أو ثلاث وظائف مختلفة لمحاولة تغطية الفواتير وم...,115,138,غالبا ما يعمل الفقراء في وظيفتين أو ثلاث وظائف...,PX6Df4I95wA,86
4,الطريقة التي يتواصل بها الشخص معك بالعين يمكن ...,186,211,الطريقة التي يتواصل بها الشخص معك بالعين يمكن ...,hVXlKj9gahs,89
...,...,...,...,...,...,...
327339,ـ ـ ـ ـ ـ,65,70,ـ ـ ـ ـ ـ,PlpxSN2qDfg,20
327340,م ن ش ي ء,1360,1363,فما أغنى عنهم سمعهم ولا أبصارهم ولا أفئدتهم مِ...,GxKz46WcZGU,100
327341,م ن ش ي ء,1493,1496,فما أغنى عنهم سمعهم ولا أبصارهم ولا أفئدتهم مِ...,GxKz46WcZGU,100
327342,ب ع ل ى غ,1861,1868,أن ينزل الله من فضله على من يشاء من عباده فباء...,QNxG9RTbDBk,100


In [30]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-30-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-30-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,227,242,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,_bfxALYv4A4
1,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,9,24,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,tEuQpz6cC0U
2,ثانيا قد تؤدي عمليات الشراء المفاجئة هذه عند و...,80,102,ثانيا قد تؤدي عمليات الشراء المفاجئة هذه عند و...,2FfcsjFbLGw
3,عملك أو العثور على الحب فإن بعض التجارب الأكثر...,106,128,سواء كان ذلك للتغلب على خجلك أو الاستثمار في ت...,DopOuLuibXU
4,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,639,658,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,hVg78Xb6KlI
...,...,...,...,...,...
277122,ا ل ي د ف,373,376,كان كلاهما خائفين ومنهكين وجدا شجرة مجوفة فدخل...,RNjqHinRK7I
277123,م ن ش ي ء,1360,1363,فما أغنى عنهم سمعهم ولا أبصارهم ولا أفئدتهم مِ...,GxKz46WcZGU
277124,م ن ش ي ء,1493,1496,فما أغنى عنهم سمعهم ولا أبصارهم ولا أفئدتهم مِ...,GxKz46WcZGU
277125,ب ع ل ى غ,1861,1868,أن ينزل الله من فضله على من يشاء من عباده فباء...,QNxG9RTbDBk


In [31]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-31-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,227,242,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,_bfxALYv4A4,100
1,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,9,24,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,tEuQpz6cC0U,100
2,ثانيا قد تؤدي عمليات الشراء المفاجئة هذه عند و...,80,102,ثانيا قد تؤدي عمليات الشراء المفاجئة هذه عند و...,2FfcsjFbLGw,88
3,عملك أو العثور على الحب فإن بعض التجارب الأكثر...,106,128,سواء كان ذلك للتغلب على خجلك أو الاستثمار في ت...,DopOuLuibXU,80
4,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,639,658,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,hVg78Xb6KlI,100
...,...,...,...,...,...,...
277122,ا ل ي د ف,373,376,كان كلاهما خائفين ومنهكين وجدا شجرة مجوفة فدخل...,RNjqHinRK7I,12
277123,م ن ش ي ء,1360,1363,فما أغنى عنهم سمعهم ولا أبصارهم ولا أفئدتهم مِ...,GxKz46WcZGU,16
277124,م ن ش ي ء,1493,1496,فما أغنى عنهم سمعهم ولا أبصارهم ولا أفئدتهم مِ...,GxKz46WcZGU,16
277125,ب ع ل ى غ,1861,1868,أن ينزل الله من فضله على من يشاء من عباده فباء...,QNxG9RTbDBk,12


In [32]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-32-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,227,242,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,_bfxALYv4A4
1,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,9,24,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,tEuQpz6cC0U
2,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,639,658,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,hVg78Xb6KlI
3,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,2572,2589,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,DiM0xqXXFcE
4,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,439,456,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,PX6Df4I95wA
...,...,...,...,...,...
109458,د ذ ر ز س ش ص,69,73,د ذ ر ز س ش ص,FYYRThrxARs
109459,أ ب ت ث ج ح خ,65,69,أ ب ت ث ج ح خ,FYYRThrxARs
109460,د ذ ر ز س ش ص,41,45,د ذ ر ز س ش ص,FYYRThrxARs
109461,و يا لك من كف,569,572,و يا لك من كف,yBWhE9LhAfk


In [33]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,بدون ملح كمان نفس النتيجة,zz48KmXXeN0
1,علينا الآن تحديد مستوى هبوط السقف المستعار,zyIre8scvbs
2,علينا الاهتمام جيدا بهذه الملاحظة,zyIre8scvbs
3,المستوى بالنسبة لهذا المكان واضح,zyIre8scvbs
4,المجموعة تخشى أنها قد حققت هدفها أخيرا ولكن حد...,zxafkDXKGQw
...,...,...
10005,نعم فقد مر وقت طويل,02qFT9oP8jU
10007,لذا فإنها غادرت للبحث عنك,02qFT9oP8jU
10008,يمكنك الذهاب لتفقد حال أخي,02qFT9oP8jU
10009,سأقوم بتسليم ختم القيادة العسكرية له الآن,02qFT9oP8jU


In [34]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,227,242,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,_bfxALYv4A4
1,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,9,24,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,tEuQpz6cC0U
2,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,639,658,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,hVg78Xb6KlI
3,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,2572,2589,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,DiM0xqXXFcE
4,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,439,456,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,PX6Df4I95wA
...,...,...,...,...,...
9908,ده فلو مش اي حد,44,46,ده فلو مش اي حد,4XabABDIPWQ
9909,د ذ ر ز س ش ص,80,85,د ذ ر ز س ش ص,tu4S707PajM
9910,د ذ ر ز س ش ص,20,24,د ذ ر ز س ش ص,FYYRThrxARs
9911,أ ب ت ث ج ح خ,16,20,أ ب ت ث ج ح خ,FYYRThrxARs


In [35]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,227,242,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,_bfxALYv4A4,https://www.youtube.com/watch?v=_bfxALYv4A4&t=...
1,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,9,24,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,tEuQpz6cC0U,https://www.youtube.com/watch?v=tEuQpz6cC0U&t=9s
2,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,639,658,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,hVg78Xb6KlI,https://www.youtube.com/watch?v=hVg78Xb6KlI&t=...
3,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,2572,2589,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,DiM0xqXXFcE,https://www.youtube.com/watch?v=DiM0xqXXFcE&t=...
4,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,439,456,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,PX6Df4I95wA,https://www.youtube.com/watch?v=PX6Df4I95wA&t=...
...,...,...,...,...,...,...
9908,ده فلو مش اي حد,44,46,ده فلو مش اي حد,4XabABDIPWQ,https://www.youtube.com/watch?v=4XabABDIPWQ&t=44s
9909,د ذ ر ز س ش ص,80,85,د ذ ر ز س ش ص,tu4S707PajM,https://www.youtube.com/watch?v=tu4S707PajM&t=80s
9910,د ذ ر ز س ش ص,20,24,د ذ ر ز س ش ص,FYYRThrxARs,https://www.youtube.com/watch?v=FYYRThrxARs&t=20s
9911,أ ب ت ث ج ح خ,16,20,أ ب ت ث ج ح خ,FYYRThrxARs,https://www.youtube.com/watch?v=FYYRThrxARs&t=16s


In [36]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [37]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [38]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [39]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Arabic_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'Arabic_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [40]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [41]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass